# Advent of code 2020: day 20

Problem [here](https://adventofcode.com/2020/day/20)

## Part 1

In [1]:
def boolsToInt(values):
    return sum(1<<i for i,v in enumerate(reversed(values)) if v)

class Tile:
    """
    Tile, storing id and edges converted to integer (normal and reversed, so 8 possibilities)
    """
    def __init__(self, uid, values):
        self.uid = uid
        self.edges = Tile.makeEdges(values)
        self.values = values
    @staticmethod
    def makeEdges(values):
        return tuple([
            boolsToInt(values[0,:]),     ## N: top row, L->R
            boolsToInt(values[:,-1]),    ## E: rightmost column, T->B
            boolsToInt(values[-1,:]),    ## S: bottom row, L->R
            boolsToInt(values[:,0]),     ## W: leftmost column, T->B
            boolsToInt(values[0,::-1]),  ## -N
            boolsToInt(values[::-1,-1]), ## -E
            boolsToInt(values[-1,::-1]), ## -S
            boolsToInt(values[::-1,0])   ## -W
        ])

import numpy as np
def readTiles(iLines):
    tiles = {}
    try:
        while True:
            hdr = next(iLines)
            assert hdr.startswith("Tile ") and hdr.endswith(":")
            tileId = int(hdr[5:-1])
            tileVals = []
            while ln := next(iLines):
                tileVals.append([ (ch == "#") for ch in ln ])
            tiles[tileId] = Tile(tileId, np.array(tileVals))
    except StopIteration:
        tiles[tileId] = Tile(tileId, np.array(tileVals))
    return tiles

with open("inputs/day20_ex1.txt") as exInF:
    ex1_tiles = readTiles((ln.strip() for ln in exInF))

In [2]:
from collections import defaultdict

def getTilesStats(tiles):
    tilesPerEdge = defaultdict(list)
    for tl in tiles:
        for ed in tl.edges:
            tilesPerEdge[ed].append(tl.uid)
    h_edgesPerTile = defaultdict(int)
    for ed,edTiles in tilesPerEdge.items():
        h_edgesPerTile[len(edTiles)] += 1
    return tilesPerEdge, h_edgesPerTile

ex1_tPerEdge, ex1_hEdgesPerTile = getTilesStats(ex1_tiles.values())
for nTiles in sorted(ex1_hEdgesPerTile.keys()):
    print(f"Number of edges present in {nTiles:d} tiles: {ex1_hEdgesPerTile[nTiles]:d}")

Number of edges present in 1 tiles: 24
Number of edges present in 2 tiles: 24


In [3]:
tileOrientations = [
    (0, 1, 2, 3), #  N E S W = leave
    (1, 6, 3, 4), #  E-S W-N = np.rot90
    (6, 7, 4, 5), # -S-W-N-E = np.rot90(, k=2)
    (7, 0, 5, 2), # -W N-E S = np.rot90(, k=-1)
    (4, 3, 6, 1), # -N W-S E = fliplr
    (5, 4, 7, 6), # -E-N-W-S = np.rot90(fliplr, k=-1)
    (2, 5, 0, 7), #  S-E N-W = flipud
    (3, 2, 1, 0)] #  W S E N = transpose
edgeOfOther = (2, 3, 0, 1) # N<->S, E<->W

def getCorners(tiles, perEdge):
    corners = []
    for tl in tiles:
        unMatched = []
        for i,ed in enumerate(tl.edges):
            tlForEd = perEdge[ed]
            if len(tlForEd) == 1:
                unMatched.append(i)
        if min(sum(1 for ie in ori if ie in unMatched) for ori in tileOrientations) == 2:
            corners.append(tl)
    return corners

ex1_corners = getCorners(ex1_tiles.values(), ex1_tPerEdge)
print(f"Corner tile(s): {', '.join(str(tl.uid) for tl in ex1_corners)}")

if len(ex1_corners) == 4:
    p = 1
    for tl in ex1_corners:
        p *= tl.uid
    print(f"Product of corner tile IDs: {p:d}")

Corner tile(s): 1951, 1171, 2971, 3079
Product of corner tile IDs: 20899048083289


In [4]:
with open("inputs/day20.txt") as inF:
    p_tiles = readTiles((ln.strip() for ln in inF))
print(len(p_tiles))
p_tPerEdge, p_hEdgesPerTile = getTilesStats(p_tiles.values())
p_corners = getCorners(p_tiles.values(), p_tPerEdge)
print(f"Corner tile(s): {', '.join(str(tl.uid) for tl in p_corners)}")
if len(p_corners) == 4:
    p = 1
    for tl in p_corners:
        p *= tl.uid
    print(f"Product of corner tile IDs: {p:d}")

144
Corner tile(s): 2221, 3011, 2251, 1847
Product of corner tile IDs: 27803643063307


## Part 2

In [5]:
def getNeighboursFilled(solution):
    nFilled = np.zeros(tuple(list(solution.shape)[:2]), dtype=np.int)
    ones = np.ones(nFilled.shape, dtype=np.int)
    zeros = np.zeros(nFilled.shape, dtype=np.int)
    nFilled[1: , : ] += np.where(solution[:-1, : ,0] != 0, ones[1:,:], zeros[1:,:])
    nFilled[:-1, : ] += np.where(solution[1: , : ,0] != 0, ones[1:,:], zeros[1:,:])
    nFilled[ : ,1: ] += np.where(solution[ : ,:-1,0] != 0, ones[:,1:], zeros[:,1:])
    nFilled[ : ,:-1] += np.where(solution[ : ,1: ,0] != 0, ones[:,1:], zeros[:,1:])
    return nFilled

def positionRequires(solution, pos, allTiles):
    i,j = pos
    requires = [None]*4
    if i != 0 and solution[i-1,j,0]: # above
        oId, oOri = solution[i-1,j]
        requires[0] = allTiles[oId].edges[tileOrientations[oOri][edgeOfOther[0]]]
    if j+1 != solution.shape[1] and solution[i,j+1,0]: # right
        oId, oOri = solution[i,j+1]
        requires[1] = allTiles[oId].edges[tileOrientations[oOri][edgeOfOther[1]]]
    if i+1 != solution.shape[0] and solution[i+1,j,0]: # above
        oId, oOri = solution[i+1,j]
        requires[2] = allTiles[oId].edges[tileOrientations[oOri][edgeOfOther[2]]]
    if j != 0 and solution[i,j-1,0]: # left
        oId, oOri = solution[i,j-1]
        requires[3] = allTiles[oId].edges[tileOrientations[oOri][edgeOfOther[3]]]
    return requires

from itertools import product

def makePuzzle(n, tiles, perEdge):
    corners = getCorners(tiles.values(), perEdge)
    startTile = corners[0]
    startTile_ors = [ i for i,ori in enumerate(tileOrientations)
                     if len(perEdge[startTile.edges[ori[0]]]) == 1
                     and len(perEdge[startTile.edges[ori[3]]]) == 1 ]
    print(f"Starting with {startTile.uid} at (0,0) in orientation {startTile_ors[0]:d}")
    solution = np.stack((np.zeros((n,n), dtype=np.int), -1*np.ones((n,n), dtype=np.int)), axis=2)
    solution[0,0] = [ startTile.uid, startTile_ors[0]]
    sol_history = [((0, 0), startTile_ors[1:])]
    usedTiles = set()
    usedTiles.add(startTile.uid)
    trialIsValid = True
    while np.any(solution[:,:,0] == 0):
        if not trialIsValid:
            while sol_history and not sol_history[-1][1]:
                (i,j), _ = sol_history[-1]
                del sol_history[-1]
                tlId = solution[i,j,0]
                solution[i,j] = [0, -1]
                usedTiles.discard(tlId)
            if not sol_history:
                raise RuntimeError("Stuck and cannot roll back")
            (i,j), otherOptions = sol_history[-1]
            solution[i,j,1] = otherOptions[0]
            sol_history[-1] = ((i,j), otherOptions[1:])
            print(f"Rolled back until position ({i:d}, {j:d}), now taking orientation {otherOptions[0]:d}")
            trialIsValid = True
        else:
            nReq = 4
            while nReq and trialIsValid:
                neighFilled = getNeighboursFilled(solution)
                if np.any(np.logical_and(neighFilled == nReq, solution[:,:,0] == 0)):
                    nFilled = 0
                    with np.nditer(neighFilled, flags=['multi_index']) as it:
                        for nN in it:
                            i,j = it.multi_index
                            if nN == nReq and solution[i,j,0] == 0:
                                req = positionRequires(solution, (i,j), tiles)
                                print(f"Checking for ({i:d},{j:d}), needs {req!r}")
                                options = []
                                for tl, (k,ori) in product(tiles.values(), enumerate(tileOrientations)):
                                    if tl.uid not in usedTiles:
                                        if all(tl.edges[i] == rq for i,rq in zip(ori,req) if rq is not None):
                                            options.append((tl.uid, k))
                                if not options:
                                    trialIsValid = False
                                    print(solution[:,:,0])
                                    print("No options, we're stuck")
                                    for tl, (k,ori) in product(tiles.values(), enumerate(tileOrientations)):
                                        if tl.uid not in usedTiles:
                                            print(f"{tl.uid:d} ori {k:d}: ({', '.join(str(tl.edges[i]) for i in ori)})")
                                    break ## a problem
                                else:
                                    nFilled += 1
                                    tlid, kO = options[0]
                                    print(f"Putting tile {tlid:d} at ({i:d},{j:d}) with orientation {kO:d} ({', '.join(str(tiles[tlid].edges[i]) for i in tileOrientations[kO])})")
                                    solution[i,j] = [ tlid, kO ]
                                    sol_history.append(((i,j), options[1:]))
                                    usedTiles.add(tlid)
                    if nFilled:
                        break # try again with more
                else:
                    nReq -= 1
    if trialIsValid:
        print(solution[:,:,0])
        return solution

ex_solution = makePuzzle(3, ex1_tiles, ex1_tPerEdge)

Starting with 1951 at (0,0) in orientation 3
Checking for (0,1), needs [None, None, None, 710]
Putting tile 2729 at (0,1) with orientation 3 (962, 85, 9, 710)
Checking for (1,0), needs [318, None, None, None]
Putting tile 2311 at (1,0) with orientation 3 (318, 210, 616, 231)
Checking for (1,1), needs [9, None, None, 210]
Putting tile 1427 at (1,1) with orientation 3 (9, 948, 348, 210)
Checking for (0,2), needs [None, None, None, 85]
Putting tile 2971 at (0,2) with orientation 3 (78, 161, 689, 85)
Checking for (1,2), needs [689, None, None, 948]
Putting tile 1489 at (1,2) with orientation 3 (689, 848, 288, 948)
Checking for (2,0), needs [616, None, None, None]
Putting tile 3079 at (2,0) with orientation 7 (616, 184, 264, 702)
Checking for (2,1), needs [348, None, None, 184]
Putting tile 2473 at (2,1) with orientation 2 (348, 399, 481, 184)
Checking for (2,2), needs [288, None, None, 399]
Putting tile 1171 at (2,2) with orientation 1 (288, 96, 902, 399)
[[1951 2729 2971]
 [2311 1427 1489

In [6]:
p_solution = makePuzzle(12, p_tiles, p_tPerEdge)

Starting with 2221 at (0,0) in orientation 2
Checking for (0,1), needs [None, None, None, 22]
Putting tile 2381 at (0,1) with orientation 7 (257, 766, 370, 22)
Checking for (1,0), needs [930, None, None, None]
Putting tile 2309 at (1,0) with orientation 3 (930, 326, 842, 567)
Checking for (1,1), needs [370, None, None, 326]
Putting tile 1997 at (1,1) with orientation 7 (370, 38, 8, 326)
Checking for (0,2), needs [None, None, None, 766]
Putting tile 3449 at (0,2) with orientation 4 (856, 224, 372, 766)
Checking for (1,2), needs [372, None, None, 38]
Putting tile 1237 at (1,2) with orientation 5 (372, 271, 153, 38)
Checking for (2,0), needs [842, None, None, None]
Putting tile 3191 at (2,0) with orientation 4 (842, 71, 249, 746)
Checking for (2,1), needs [8, None, None, 71]
Putting tile 3947 at (2,1) with orientation 4 (8, 352, 770, 71)
Checking for (2,2), needs [153, None, None, 352]
Putting tile 1307 at (2,2) with orientation 5 (153, 1016, 218, 352)
Checking for (0,3), needs [None, Non


Checking for (8,8), needs [672, None, None, 553]
Putting tile 3499 at (8,8) with orientation 7 (672, 135, 679, 553)
Checking for (0,9), needs [None, None, None, 319]
Putting tile 1087 at (0,9) with orientation 3 (476, 367, 921, 319)
Checking for (1,9), needs [921, None, None, 832]
Putting tile 3061 at (1,9) with orientation 6 (921, 580, 274, 832)
Checking for (2,9), needs [274, None, None, 243]
Putting tile 2833 at (2,9) with orientation 3 (274, 300, 572, 243)
Checking for (3,9), needs [572, None, None, 889]
Putting tile 3637 at (3,9) with orientation 4 (572, 195, 965, 889)
Checking for (4,9), needs [965, None, None, 691]
Putting tile 3467 at (4,9) with orientation 7 (965, 560, 1020, 691)
Checking for (5,9), needs [1020, None, None, 587]
Putting tile 3607 at (5,9) with orientation 2 (1020, 487, 851, 587)
Checking for (6,9), needs [851, None, None, 517]
Putting tile 2293 at (6,9) with orientation 7 (851, 537, 1013, 517)
Checking for (7,9), needs [1013, None, None, 720]
Putting tile 233


Checking for (10,8), needs [683, None, None, 518]
Putting tile 3929 at (10,8) with orientation 0 (683, 689, 373, 518)
Checking for (10,9), needs [546, None, None, 689]
Putting tile 1487 at (10,9) with orientation 5 (546, 456, 712, 689)
Checking for (10,10), needs [323, None, None, 456]
Putting tile 1831 at (10,10) with orientation 5 (323, 602, 492, 456)
Checking for (0,11), needs [None, None, None, 209]
Putting tile 2251 at (0,11) with orientation 7 (123, 858, 588, 209)
Checking for (1,11), needs [588, None, None, 519]
Putting tile 3613 at (1,11) with orientation 7 (588, 32, 762, 519)
Checking for (2,11), needs [762, None, None, 707]
Putting tile 2593 at (2,11) with orientation 2 (762, 217, 753, 707)
Checking for (3,11), needs [753, None, None, 959]
Putting tile 3089 at (3,11) with orientation 2 (753, 722, 800, 959)
Checking for (4,11), needs [800, None, None, 721]
Putting tile 3319 at (4,11) with orientation 5 (800, 246, 742, 721)
Checking for (5,11), needs [742, None, None, 547]
Put

In [7]:
oriTransforms = [
    lambda a : a,                            #  N E S W = leave
    lambda a : np.rot90(a),                  #  E-S W-N = np.rot90
    lambda a : np.rot90(a, k=2),             # -S-W-N-E = np.rot90(, k=2)
    lambda a : np.rot90(a, k=-1),            # -W N-E S = np.rot90(, k=-1)
    lambda a : np.fliplr(a),                 # -N W-S E = fliplr
    lambda a : np.rot90(np.fliplr(a), k=-1), # -E-N-W-S = np.rot90(fliplr, k=-1)
    lambda a : np.flipud(a),                 #  S-E N-W = flipud
    lambda a : np.transpose(a)]              #  W S E N = transpose

m = np.array([ [ 1., 2., 3. ], [ 4., 5., 6. ], [ 7., 8., 9. ] ])
for i,t in enumerate(oriTransforms):
    print(i, t(m))

0 [[1. 2. 3.]
 [4. 5. 6.]
 [7. 8. 9.]]
1 [[3. 6. 9.]
 [2. 5. 8.]
 [1. 4. 7.]]
2 [[9. 8. 7.]
 [6. 5. 4.]
 [3. 2. 1.]]
3 [[7. 4. 1.]
 [8. 5. 2.]
 [9. 6. 3.]]
4 [[3. 2. 1.]
 [6. 5. 4.]
 [9. 8. 7.]]
5 [[9. 6. 3.]
 [8. 5. 2.]
 [7. 4. 1.]]
6 [[7. 8. 9.]
 [4. 5. 6.]
 [1. 2. 3.]]
7 [[1. 4. 7.]
 [2. 5. 8.]
 [3. 6. 9.]]


In [8]:
ex1_full = [ [ oriTransforms[ori](ex1_tiles[uid].values) for uid,ori in row ] for row in ex_solution ]
p_full = [ [ oriTransforms[ori](p_tiles[uid].values) for uid,ori in row ] for row in p_solution ]

In [9]:
def getLineStr(arr):
    return [ "".join(("#" if ch else ".") for ch in row) for row in arr ]

for i,row in enumerate(ex1_full):
    for linePerBlock in zip(*(getLineStr(tile) for tile in row)):
        print(" ".join(linePerBlock))
    print("")

#..#..#.## ####....#. ...#..###.


..####.... .#####..#. ....###...
.#####..## #..#.#.##. .##..#.#.#
..#.#...## #.###...## #.##.##...
##.#.##.#. .##.#.##.. .##.######
#..##.###. ..####..## #.####.##.
....#.#... .##.##.... .#..#..##.
##.##.#..# ##.#..#..# ###.#.#...
..###.##.# #....#.... .###.#....
.#..#####. ......#..# #.#.##...#

.#..#####. ......#..# #.#.##...#
.#.####.#. ..#....### #.#..#.#.#
###...#..# ##....#..# ##...####.
#..#.##..# #.###..#.. .#####..##
#....#.##. ..#.###### #....#....
...##.##.# ####.....# ##.##..#.#
.#...#.... .###..###. .#....##..
#.#.##.... .#.##.#.## #.###...#.
##.###.#.# #.####.... .##..#....
#..##.#... .#.#.###.. .#..#.....

#..##.#... .#.#.###.. .#..#.....
.#.###.... .#.##...## #.######..
#.###.#### #.######## #..#####..
...##.#... .#..#.###. .####.####
##.#..##.# #########. ...#..##.#
##.#####.# #.#.#...#. .#..#.....
##....##.# #.#.###### #####..##.
##...#.... ...#..##.# #..###.##.
##..###... ...##.#..# #.##.##.#.
.#....#... .####....# ###....##.



In [10]:
ex_merged = np.vstack((np.hstack((vals[1:-1,1:-1] for vals in row)) for row in ex1_full)).T
print("\n".join("".join(("#" if ch else ".") for ch in row) for row in ex_merged))

.#.#..#.##...#.##..#####
###....#.#....#..#......
##.##.###.#.#..######...
###.#####...#.#####.#..#
##.#....#.##.####...#.##
...########.#....#####.#
....#..#...##..#.#.###..
.####...#..#.....#......
#..#.##..#..###.#.##....
#.####..#.####.#.#.###..
###.#.#...#.######.#..##
#.####....##..########.#
##..##.#...#...#.#.#.#..
...#..#..#.#.##..###.###
.#.#....#.##.#...###.##.
###.#...#..#.##.######..
.#.#.###.##.##.#..#.##..
.####.###.#...###.#..#.#
..#.#..#..#.#.#.####.###
#..####...#.#.#.###.###.
#####..#####...###....##
#.##..#..#...#..####...#
.#.###..##..##..####.##.
...###...##...#...#..###


<ipython-input-1-b9b9daeccc52>:1: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  ex_merged = np.vstack((np.hstack((vals[1:-1,1:-1] for vals in row)) for row in ex1_full)).T


In [11]:
p_merged = np.vstack((np.hstack((vals[1:-1,1:-1] for vals in row)) for row in p_full)).T

<ipython-input-1-df8e425207ec>:1: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  p_merged = np.vstack((np.hstack((vals[1:-1,1:-1] for vals in row)) for row in p_full)).T


In [12]:
monster_pat = np.array([ [(True if ch == "#" else False) for ch in row] for row in """                  # 
#    ##    ##    ###
 #  #  #  #  #  #   """.split("\n") ], dtype=np.bool)

In [13]:
ex_nMonsters = 0
ex_monsters = []
for k,tr in enumerate(oriTransforms):
    t_m = tr(monster_pat)
    for i in range(ex_merged.shape[0]-t_m.shape[0]):
        for j in range(ex_merged.shape[1]-t_m.shape[1]):
            if np.all(t_m == np.logical_and(ex_merged[i:i+t_m.shape[0], j:j+t_m.shape[1]], t_m)):
                ex_monsters.append(((i,j), k))
                ex_nMonsters += 1
print(f"Example monsters: {ex_nMonsters:d}, {ex_monsters!r}")
ex_forRough = np.array(ex_merged)
for (i,j), k in ex_monsters:
    t_m = oriTransforms[k](monster_pat)
    ex_forRough[i:i+t_m.shape[0], j:j+t_m.shape[1]] = np.where(
        np.logical_and(ex_forRough[i:i+t_m.shape[0], j:j+t_m.shape[1]], t_m),
        np.full(t_m.shape, False), ex_forRough[i:i+t_m.shape[0], j:j+t_m.shape[1]])
print(f"Example roughness: {np.sum(ex_forRough):d}")

Example monsters: 2, [((1, 16), 7), ((2, 2), 7)]
Example roughness: 273


In [14]:
p_nMonsters = 0
p_monsters = []
for k,tr in enumerate(oriTransforms):
    t_m = tr(monster_pat)
    for i in range(p_merged.shape[0]-t_m.shape[0]):
        for j in range(p_merged.shape[1]-t_m.shape[1]):
            if np.all(t_m == np.logical_and(p_merged[i:i+t_m.shape[0], j:j+t_m.shape[1]], t_m)):
                p_monsters.append(((i,j), k))
                p_nMonsters += 1
print(f"Puzzle monsters: {p_nMonsters:d}, {p_monsters!r}")
p_forRough = np.array(p_merged)
for (i,j), k in p_monsters:
    t_m = oriTransforms[k](monster_pat)
    p_forRough[i:i+t_m.shape[0], j:j+t_m.shape[1]] = np.where(
        np.logical_and(p_forRough[i:i+t_m.shape[0], j:j+t_m.shape[1]], t_m),
        np.full(t_m.shape, False), p_forRough[i:i+t_m.shape[0], j:j+t_m.shape[1]])
print(f"Puzzle roughness: {np.sum(p_forRough):d}")

Puzzle monsters: 37, [((2, 71), 4), ((3, 7), 4), ((3, 32), 4), ((7, 65), 4), ((8, 21), 4), ((13, 43), 4), ((14, 15), 4), ((15, 70), 4), ((21, 55), 4), ((24, 18), 4), ((27, 60), 4), ((31, 26), 4), ((32, 57), 4), ((36, 2), 4), ((37, 70), 4), ((38, 42), 4), ((42, 7), 4), ((43, 30), 4), ((46, 52), 4), ((47, 23), 4), ((52, 34), 4), ((53, 5), 4), ((54, 70), 4), ((59, 58), 4), ((61, 35), 4), ((62, 8), 4), ((65, 67), 4), ((69, 23), 4), ((70, 60), 4), ((74, 40), 4), ((75, 13), 4), ((76, 63), 4), ((80, 10), 4), ((80, 58), 4), ((85, 16), 4), ((86, 67), 4), ((90, 8), 4)]
Puzzle roughness: 1644
